In [78]:
import os
import re
import json



In [2]:
def search_files(directory, includes=[], excludes=[], type="file") -> list:
    """
    :Param
    :type: dir | file
    """
    file_list = []
    for file in os.listdir(directory):
        if all(include in file for include in includes) and all(exclude not in file for exclude in excludes):
            absolute_file = os.path.join(directory, file)
            if type == "file" and os.path.isfile(absolute_file):
                file_list.append(absolute_file)
            if type == "dir" and os.path.isdir(absolute_file):
                file_list.append(absolute_file)
    return file_list

In [3]:

filenames = search_files("/home/anichikage/workspace/dram/dram-failure-prediction/data/staging", includes=["ce_dmesg"])

In [4]:
print(filenames)

['/home/anichikage/workspace/dram/dram-failure-prediction/data/staging/3.5.2-10.200.105.1-ce_dmesg-20240221.log', '/home/anichikage/workspace/dram/dram-failure-prediction/data/staging/3.5.2-10.200.104.113-ce_dmesg-20240221.log', '/home/anichikage/workspace/dram/dram-failure-prediction/data/staging/3.5.2-10.200.104.111-ce_dmesg-20240221.log', '/home/anichikage/workspace/dram/dram-failure-prediction/data/staging/huabei3-10.3.0.121-ce_dmesg-20240221.log', '/home/anichikage/workspace/dram/dram-failure-prediction/data/staging/huabei3-10.3.0.122-ce_dmesg-20240221.log']


In [5]:


def check_one(filename):
    data_list = []
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "row" in line and "col" in line and "ba" in line and "bg" in line and "rank" in line:
                d = {}
                pattern = r"socket:(\d+) imc:(\d+) rank:(\d+) bg:(\d+) ba:(\d+) row:(\w+) col:(\w+)"
                match = re.search(pattern, line)
                if match:
                    d["socket"] = match.group(1)
                    d["imc"] = match.group(2)
                    d["rank"] = match.group(3)
                    d["bg"] = match.group(4)
                    d["ba"] = match.group(5)
                    d["row"] = match.group(6)
                    d["col"] = match.group(7)
                    data_list.append(d)
    return data_list
    

In [10]:


for filename in filenames:
    # filename = "/home/anichikage/workspace/dram/dram-failure-prediction/data/staging/10.200.104.113.dmesg.log"
    # print(filename)
    data = check_one(filename)

    print(len(data))

1749
1415
1153
220
246


In [7]:
data[0]

{'socket': '1',
 'imc': '1',
 'rank': '1',
 'bg': '3',
 'ba': '1',
 'row': '5b12',
 'col': '2b0'}

### ce

In [8]:
filenames = search_files("/home/anichikage/workspace/dram/dram-failure-prediction/data/staging/3.5.2-10.200.104.111-ce_dmesg-20240221.log", includes=["log"])

NotADirectoryError: [Errno 20] Not a directory: '/home/anichikage/workspace/dram/dram-failure-prediction/data/staging/3.5.2-10.200.104.111-ce_dmesg-20240221.log'

In [72]:
import numpy as np

max_row = 131072
max_col = 1024
row_block = 32
col_block = 16
row_section_size = max_row // row_block
col_section_size = max_col // col_block

ce_list = []

def hex_to_decimal(hex_string):
    decimal_number = int(hex_string, 16)
    return decimal_number

def parse_line(line):
    pattern = r'row:(\w+) col:(\w+)'
    match = re.search(pattern, line)
    if match:
        row = match.group(1)
        col = match.group(2)
    return hex_to_decimal(row), hex_to_decimal(col)

ce_slices = []

for filename in filenames:
    ce_slice = []
    time_slice = []
    host_matrix = np.zeros([row_block, col_block])
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "row" in line and "col" in line and "bg" in line:
                row, col = parse_line(line)
                host_matrix[int(row // row_section_size), int(col // col_section_size)] += 1
                # data = np.load(host_matrix).astype(np.float32)
                data = 2 / (1 + np.exp(-host_matrix)) - 1
                ce_slice.append(host_matrix)
    # print(len(ce_slice))
    # print(ce_slice[50])
    for i in range(len(ce_slice)):
        ce_slice[i] = 2 / (1 + np.exp(-ce_slice[i])) - 1
    # print(ce_slice[50])
    ce_slices.append(ce_slice)    

In [73]:


for ce_slice in ce_slices:
    pass
    

In [70]:
time_slice = ce_slices[0][100]

In [71]:
time_slice

array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        ],
       [0.9981779 , 0.96402758, 0.46211716, 0.9993293 , 0.90514825,
        0.        , 0.90514825, 0.90514825, 0.99505475, 0.9866143 ,
        0.90514825, 0.96402758, 0.96402758, 0.46211716, 0.99505475,
        0.9866143 ],
       [0.90514825, 0.90514825, 0.96402758, 0.9981779 , 0.76159416,
        0.90514825, 0.90514825, 0.90514825, 0.9866143 , 0.76159416,
        0.9866143 , 0.90514825, 0.90514825, 0.        , 0.76159416,
        0.        ],
       [0.9866143 , 0.99505475, 0.76159416, 0.46211716, 0.46211716,
        0.9866143 , 0.99505475, 0.90514825, 0.9866143 , 0.90514825,
        0.76159416, 0.96402758, 0.76159416, 0.9981779 , 0.9866143 ,
        0.9866143 ],
       [0.76159416, 0.9866143 , 0.76159416, 0.90514825, 0.90514825,
        0.96402758, 0.        , 

In [15]:
import torch

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.utils.data import DataLoader, TensorDataset, Dataset, SubsetRandomSampler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from gradcam import GradCAM
import matplotlib.pyplot as plt

import model as md
import config as CONFIG
import utils

In [17]:


model = md.ResNet4_32x16(conv1_kernel_size=9, conv2_kernel_size=5, \
                                  conv3_kernel_size=3, conv4_kernel_size=3)
model.load_state_dict(torch.load(os.path.join(CONFIG.PATH_MODEL, "resnet-ConvRule-model-0.1.pth")))

<All keys matched successfully>

In [19]:
model.to(torch.device("cpu"))
model.eval()

ResNet4_32x16(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicResNetBlock(
      (conv1): Conv2d(32, 32, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicResNetBlock(
      (conv1): Conv2d(32, 32, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(32, 32, kernel_siz

In [59]:
device = torch.device("cpu")

batch_size = 1

real_data = np.array([[time_slice]]).astype(np.float32)
real_label = np.array([0])

real_dataset = utils.CustomDataset(real_data, real_label)
real_loader = DataLoader(real_dataset, batch_size=batch_size, shuffle=False)


In [79]:
real_data.shape

(1, 1, 32, 16)

In [80]:
real_label.shape

(1,)

In [60]:
feats = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"train_feats_32x16.npy")).astype(np.float32)

feats.shape

(2873, 1, 32, 16)

In [61]:
labels = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"train_labels_32x16.npy")).astype(np.float32)

labels.shape

labels

array([0., 1., 0., ..., 0., 0., 0.], dtype=float32)

In [62]:
real_data.shape
real_label.shape

(1,)

In [66]:


for batch in real_loader:
    inputs, labels = batch["data"].to(device), batch["label"].to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    print(predicted)

tensor([0])


In [67]:
predicted

tensor([0])

In [76]:
from tqdm import tqdm

for ce_slice_one_node in ce_slices:
    ces = []
    for ce in tqdm(ce_slice_one_node):
        real_data = np.array([[ce]]).astype(np.float32)
        real_label = np.array([0])

        real_dataset = utils.CustomDataset(real_data, real_label)
        real_loader = DataLoader(real_dataset, batch_size=batch_size, shuffle=False)

        for batch in real_loader:
            inputs, labels = batch["data"].to(device), batch["label"].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            if predicted == 1:
                ces.append(ce)
                
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1749/1749 [00:16<00:00, 103.62it/s]


In [77]:
len(ces)

0